In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def roundTime(dt=None):    
    dt = dt.to_pydatetime()
    roundTo = 15*60    
    if dt == None : dt = datetime.datetime.now()
    seconds = (dt.replace(tzinfo=None) - dt.min).seconds
    rounding = (seconds+roundTo/2) // roundTo * roundTo
    return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)


def conv_entries(arr):  
    int_arr = [int(x) for x in arr]
    if int_arr[2] > 29:
        int_arr[1] += 1        
    int_arr[2] = 0
    mins = int_arr[0]*60 + int_arr[1]
    rem = mins % 15
    if rem > 7:
        mins += (15-rem)
    else:
        mins -= rem        
    return mins

def get_day_index(ts):
    idx = ts.weekday()
    if idx == 6:
        return 0
    return idx+1

def conv_timestamp(ts):
    time_arr = str(ts)[-8:].split(':')
    hours = int(time_arr[0])
    mins = int(time_arr[1])
    return (hours*60+mins)//15

def get_weekend(ts):
    if ts.weekday() > 4:
        return 1
    return 0

# def get_weekday(ts):
#     return ts.weekday()
    

def calc_quarter_load(row):
    dur = row['Charging_Time__hh_mm_ss_']
    total = row['Energy__kWh_']
    return (total/dur)*15.0


def add_to_backbones(row, stat_name):
    delta = row['End_Date___Time'] - row['Start_Date___Time']
    iters = int(round(delta.total_seconds()/60.0)/15.0)


    backbone_load = stat_backbones[stat_name][0]
    for i in range(iters):
        backbone_load.loc[backbone_load['date_time'] == row['Start_Date___Time'] + datetime.timedelta(minutes=15*i), 'value'] += row['Load_per_quarter']     
    
    backbone_occup = stat_backbones[stat_name][1]
    for i in range(iters):
        backbone_occup.loc[backbone_occup['date_time'] == row['Start_Date___Time'] + datetime.timedelta(minutes=15*i), 'value'] = 1.0   
          

In [ ]:
locations = ['boulder', 'palo_alto', 'acn', 'perth_kinross']